In [5]:
import openai, json

client = openai.OpenAI()
messages = []

In [ ]:
def get_weather(city):
    return "33 degrees celcius"


FUNCTION_MAP = {"get_weather": get_weather}

In [3]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The name of the city to get the weather of.",
                        }
                    },
                },
                "required": ["city"],
            },
        },
    }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message):
    if message.tool_calls > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function {function_name} with  {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            messages.append(
                {
                    "role": "function",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

In [ ]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4.1-nano", messages=messages, tools=TOOLS
    )
    process_ai_response(response.choices[0].message)
    print(response)
    message = response.choices[0].message.content
    messages.append

    ({"role": "assistant", "content": message})
    print(f"AI: {message}")

In [ ]:
while True:
    message = input("Send a message to the LLM....")
    if message == "quit" or message == "q":
        break
    else:

        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()